In [1]:
!pip install -q transformers datasets

In [2]:
from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
dataset = load_dataset("LucasThil/miniwobplusplus_train")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating validate split:   0%|          | 0/81548 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/81549 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/652385 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/LucasThil___parquet/LucasThil--miniwobplusplus_train-d9aac14f4796bdf9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    validate: Dataset({
        features: ['episodes', 'actions', 'refs'],
        num_rows: 81548
    })
    test: Dataset({
        features: ['episodes', 'actions', 'refs'],
        num_rows: 81549
    })
    train: Dataset({
        features: ['episodes', 'actions', 'refs'],
        num_rows: 652385
    })
})

In [5]:
example = dataset['train'][0]
example

{'episodes': '{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: click, ref: 15}{action: click, ref: 15}{action: mouseup, ref: 15}{action: mouseup, ref: 15}{action: mousedown, ref: 1}{action: mousedown, ref: 1}{action: scroll, ref: 1} {"Departure City":"Miami","Destination City":"Pittsburgh","Ticket Type":"One-way","Departure Day":29,"Passengers":3,"Seat type":"Coach"} <BODY ref="1"><A classes="banner" ref="2"><IMG classes="alaska_logo" ref="3"></IMG></A><DIV id="main" ref="4"><DIV id="starth1" classes="header" ref="5"><H1 classes="h1" ref="6" text="Book a flight"></H1></DIV><FORM id="searchForm" classes="miniwob-main-form" ref="7"><DIV classes="form-row group" ref="8">

Create a list that contains the labels, as well as two dictionaries that map labels to integers and back

In [6]:
# Have unified list for labels
chars_actions = sorted(list(set(dataset['train']['actions'] + dataset['test']['actions'] + dataset['validate']['actions'])))
chars_ref = sorted(list(set(dataset['train']['refs'] + dataset['test']['refs'] + dataset['validate']['refs'])))
labels = chars_actions + chars_ref
vocab_size = len(labels)

id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

print(labels)

['click', 'dblclick', 'keydown', 'keypress', 'keyup', 'mousedown', 'mouseup', 'scroll', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 185, 186, 187, 188, 189, 190, 191, 192, 195, 197, 198, 199, 200, 201, 202, 203, 205, 206, 207, 208, 209, 

In [20]:
print(id2label)
print(label2id)

{0: 'click', 1: 'dblclick', 2: 'keydown', 3: 'keypress', 4: 'keyup', 5: 'mousedown', 6: 'mouseup', 7: 'scroll', 8: 1, 9: 2, 10: 3, 11: 4, 12: 5, 13: 6, 14: 7, 15: 8, 16: 9, 17: 10, 18: 11, 19: 12, 20: 13, 21: 14, 22: 15, 23: 16, 24: 17, 25: 18, 26: 19, 27: 20, 28: 21, 29: 22, 30: 23, 31: 24, 32: 25, 33: 26, 34: 27, 35: 28, 36: 29, 37: 30, 38: 31, 39: 32, 40: 33, 41: 34, 42: 35, 43: 36, 44: 37, 45: 38, 46: 39, 47: 40, 48: 41, 49: 42, 50: 43, 51: 44, 52: 45, 53: 46, 54: 47, 55: 48, 56: 49, 57: 50, 58: 51, 59: 52, 60: 53, 61: 54, 62: 55, 63: 56, 64: 57, 65: 58, 66: 59, 67: 60, 68: 61, 69: 62, 70: 63, 71: 64, 72: 65, 73: 66, 74: 67, 75: 68, 76: 69, 77: 70, 78: 71, 79: 72, 80: 73, 81: 74, 82: 75, 83: 76, 84: 77, 85: 78, 86: 79, 87: 80, 88: 81, 89: 82, 90: 83, 91: 84, 92: 85, 93: 86, 94: 87, 95: 88, 96: 89, 97: 90, 98: 91, 99: 92, 100: 93, 101: 94, 102: 95, 103: 96, 104: 97, 105: 98, 106: 99, 107: 100, 108: 101, 109: 102, 110: 103, 111: 104, 112: 105, 113: 106, 114: 107, 115: 108, 116: 109, 

# Old stoi itos:

In [5]:
labels = [label for label in dataset['train'].features.keys() if label not in ['episodes']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

labels

['actions', 'refs']

In [6]:
print(id2label)
print(label2id)

{0: 'actions', 1: 'refs'}
{'actions': 0, 'refs': 1}


In [13]:
# The set function gets the set of all characters. Then make it a list and sort.
#This is our vocabulary
# PREPARE FOR ACTIONS
chars = sorted(list(set(dataset['train']['actions'] + dataset['test']['actions'] + dataset['validate']['actions'])))
vocab_size = len(chars)
print(', '.join(chars))
print(vocab_size)

# Let's build both the encoder and the decoder of the tokenization
# Create a mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
print(stoi)
#encode_action = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of intergers
#decode_action = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

#print(encode_action("click"))
#print(decode_action(encode_action("keydown")))

click, dblclick, keydown, keypress, keyup, mousedown, mouseup, scroll
8
{'click': 0, 'dblclick': 1, 'keydown': 2, 'keypress': 3, 'keyup': 4, 'mousedown': 5, 'mouseup': 6, 'scroll': 7}


In [125]:
print(stoi.dtype)

AttributeError: ignored

In [14]:
chars_ref = sorted(list(set(dataset['train']['refs'] + dataset['test']['refs'] + dataset['validate']['refs'])))
vocab_size_ref = len(chars_ref)
#print(', '.join(chars_ref))
print(vocab_size_ref)

# Let's build both the encoder and the decoder of the tokenization
# Create a mapping from characters to integers
stoi_ref = {ch:i for i, ch in enumerate(chars_ref)}
itos_ref = {i:ch for i, ch in enumerate(chars_ref)}
print(stoi_ref)

893
{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 55, 57: 56, 58: 57, 59: 58, 60: 59, 61: 60, 62: 61, 63: 62, 64: 63, 65: 64, 66: 65, 67: 66, 68: 67, 69: 68, 70: 69, 71: 70, 72: 71, 73: 72, 74: 73, 75: 74, 76: 75, 77: 76, 78: 77, 79: 78, 80: 79, 81: 80, 82: 81, 83: 82, 84: 83, 85: 84, 86: 85, 87: 86, 88: 87, 89: 88, 90: 89, 91: 90, 92: 91, 93: 92, 94: 93, 95: 94, 96: 95, 97: 96, 98: 97, 99: 98, 100: 99, 101: 100, 102: 101, 103: 102, 104: 103, 105: 104, 106: 105, 107: 106, 108: 107, 109: 108, 110: 109, 111: 110, 112: 111, 113: 112, 114: 113, 115: 114, 116: 115, 117: 116, 118: 117, 119: 118, 120: 119, 121: 120, 122

In [43]:
# try concatenate test
action_ex = dataset['train'][0]['actions']
print(action_ex)
ref_ex = dataset['train'][0]['refs']
print(ref_ex)

# encode
action_encode = stoi[action_ex]
print(action_encode)
ref_encode = stoi_ref[ref_ex]
print(ref_encode)

# right thing to do?
# ==> We should basicaly concatenate both dicts together I think
concatenated_label = str(action_encode) + str(ref_encode)
print(concatenated_label)

keydown
15
2
14
214


In [15]:
print(labels)
# put them all on one list: actions + refs
labels = list(stoi.keys()) + list(stoi_ref.keys())
print(labels)
print(len(labels))

['click', 'dblclick', 'keydown', 'keypress', 'keyup', 'mousedown', 'mouseup', 'scroll', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 185, 186, 187, 188, 189, 190, 191, 192, 195, 197, 198, 199, 200, 201, 202, 203, 205, 206, 207, 208, 209, 

In [ ]:
# try to create itos and stoi with all possible labels
mega_chars = 

# Preprocess Data

todo: we need to define the max_length and produce the snippets. So create a function that makes such snippets. It has to contain all the refs needed to perform the necessary actions.

In [38]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  print(examples[:1000])
  # take a batch of texts
  # explore preprocess_data
  # take a batch of texts
  # explore preprocess_data
  text = examples["episodes"]
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=3731)
  print(f'examples_key {examples.keys()}')
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  print(f'labels_batch: {labels_batch}')

  # make example['action'] and example['ref'] as a one-hot vector
  #labels_batch = np.zeros(vocab_size + vocab_size_ref)
  #labels_batch[stoi[examples['actions']]] = 1
  #labels_batch[stoi_ref[examples['refs']]] = 1

  labels_matrix = np.zeros((len(text), len(labels)))
  #actions_ex = examples['actions']
  #refs_ex = examples['refs']


  #for action in actions_ex:
  #  labels_matrix[:, label2id[action]] = 1
  
  #for ref in refs_ex:
  #  labels_matrix[:, label2id[ref]] = 1

  for idx, label in enumerate(labels):
    print(label)
    id = label2id[label]
    print(f'id2label: {id}')
    print(f'labels_batch[id]: {labels_batch[id]}')
    labels_matrix[:, idx] = labels_batch[id]


  encoding["labels"] = labels_matrix.tolist()

  return encoding

# Old Dissection:

In [69]:
print(len(dataset['train'][0]['episodes']))

3731


In [118]:
# explore preprocess_data
example = dataset['train'][0]
print(example)

# take a batch of texts
text = example["episodes"]
print(text)

print(example['actions'])
encoding = tokenizer(text, padding="max_length", truncation=True, max_length=3731)
print(f'encoding: {encoding}')

# make example['action'] and example['ref'] as a one-hot vector
labels_batch = np.zeros(vocab_size + vocab_size_ref)
labels_batch[stoi[example['actions']]] = 1
labels_batch[stoi_ref[example['refs']]] = 1

print(f'labels_batch: {labels_batch}')

labels_matrix = np.zeros((len(text), len(labels)))
print(labels_matrix.shape)

labels_matrix[:, stoi[example['actions']]] = 1
labels_matrix[:, stoi_ref[example['refs']]] = 1

#for idx, label in enumerate(labels):
#  print(label)
#  print(labels_batch[label])
#  labels_matrix[:, idx] = labels_batch[label]

encoding["labels"] = labels_matrix.tolist()
print(encoding['input_ids'])

{'episodes': '{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: click, ref: 15}{action: click, ref: 15}{action: mouseup, ref: 15}{action: mouseup, ref: 15}{action: mousedown, ref: 1}{action: mousedown, ref: 1}{action: scroll, ref: 1} {"Departure City":"Miami","Destination City":"Pittsburgh","Ticket Type":"One-way","Departure Day":29,"Passengers":3,"Seat type":"Coach"} <BODY ref="1"><A classes="banner" ref="2"><IMG classes="alaska_logo" ref="3"></IMG></A><DIV id="main" ref="4"><DIV id="starth1" classes="header" ref="5"><H1 classes="h1" ref="6" text="Book a flight"></H1></DIV><FORM id="searchForm" classes="miniwob-main-form" ref="7"><DIV classes="form-row group" ref="8">

# Encoding

In [36]:
# Get smaller dataset
sub_train = dataset['train'][:200]
sub_test = dataset['test'][:40]
sub_validate = dataset['validate'][:40]
sub_dataset = dataset#.clone()
sub_dataset['train'] = sub_train
sub_dataset['test'] = sub_test
sub_dataset['validate'] = sub_validate

In [39]:
encoded_dataset2 = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)
#encoded_dataset = sub_dataset.map(preprocess_data, batched=True, remove_columns=sub_dataset['train'].column_names)

  0%|          | 0/82 [00:00<?, ?ba/s]

TypeError: ignored

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# Save encoded dataset somewhere for convenience
import pandas as pd
df = pd.DataFrame(encoded_dataset)


ValueError: ignored

In [12]:
print(encoded_dataset['train'][0])

{'input_ids': [101, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2321, 1065, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2321, 1065, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2321, 1065, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2321, 1065, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2321, 1065, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2321, 1065, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2321, 1065, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2321, 1065, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2321, 1065, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2321, 1065, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2321, 1065, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2321, 1065, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2321, 1065, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2321, 1065, 1063, 2895, 1024, 11562, 1010, 25416, 1024, 2321, 1065, 1063, 2895, 1024, 11562, 1010, 25416, 1024, 2321, 1065, 1063, 2895, 10

In [15]:
example = encoded_dataset['train'][0]
print(example.keys())
print(example['labels'])
print(example['input_ids'])

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0

In [16]:
tokenizer.decode(example['input_ids'])

'[CLS] { action : keydown, ref : 15 } { action : keydown, ref : 15 } { action : keydown, ref : 15 } { action : keydown, ref : 15 } { action : keydown, ref : 15 } { action : keydown, ref : 15 } { action : keydown, ref : 15 } { action : keydown, ref : 15 } { action : keydown, ref : 15 } { action : keydown, ref : 15 } { action : keydown, ref : 15 } { action : keydown, ref : 15 } { action : keydown, ref : 15 } { action : keydown, ref : 15 } { action : click, ref : 15 } { action : click, ref : 15 } { action : mouseup, ref : 15 } { action : mouseup, ref : 15 } { action : mousedown, ref : 1 } { action : mousedown, ref : 1 } { action : scroll, ref : 1 } { " departure city " : " miami ", " destination city " : " pittsburgh ", " ticket type " : " one - way ", " departure day " : 29, " passengers " : 3, " seat type " : " coach " } < body ref = " 1 " > < a classes = " banner " ref = " 2 " > < img classes = " alaska _ logo " ref = " 3 " > < / img > < / a > < div id = " main " ref = " 4 " > < div id

In [17]:
print(encoded_dataset['train'][4])

{'input_ids': [101, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2340, 1065, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2340, 1065, 1063, 2895, 1024, 3145, 6279, 1010, 25416, 1024, 2340, 1065, 1063, 2895, 1024, 3145, 6279, 1010, 25416, 1024, 2340, 1065, 1063, 2895, 1024, 3145, 20110, 1010, 25416, 1024, 2340, 1065, 1063, 2895, 1024, 3145, 20110, 1010, 25416, 1024, 2340, 1065, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2340, 1065, 1063, 2895, 1024, 3145, 7698, 1010, 25416, 1024, 2340, 1065, 1063, 2895, 1024, 11562, 1010, 25416, 1024, 2340, 1065, 1063, 2895, 1024, 11562, 1010, 25416, 1024, 2340, 1065, 1063, 2895, 1024, 8000, 6279, 1010, 25416, 1024, 2340, 1065, 1063, 2895, 1024, 8000, 6279, 1010, 25416, 1024, 2340, 1065, 1063, 2895, 1024, 8000, 7698, 1010, 25416, 1024, 1021, 1065, 1063, 2895, 1024, 8000, 7698, 1010, 25416, 1024, 1021, 1065, 4607, 2340, 1024, 2340, 7610, 2004, 1996, 2051, 1998, 2811, 12040, 1012, 1026, 2303, 25416, 1027, 1000, 1021, 1000, 1028, 1026, 4487, 

In [18]:
example['labels']

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0

In [28]:
[itos[idx] for idx, label in enumerate(example['labels'][:vocab_size]) if label == 1.0]

['click',
 'dblclick',
 'keydown',
 'keypress',
 'keyup',
 'mousedown',
 'mouseup',
 'scroll']

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

In [27]:
encoded_dataset.set_format("torch")

# Define Model